<a href="https://colab.research.google.com/github/Alenushka2013/Projects/blob/main/Task_Skelar_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipython-sql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00


In [2]:
%load_ext sql
%sql sqlite:///my_database.db

import prettytable
prettytable.DEFAULT = prettytable.PLAIN_COLUMNS

<ipython-input-2-01f69f483b63>:5: DeprecationWarning: the 'PLAIN_COLUMNS' constant is deprecated, use the 'TableStyle' enum instead
  prettytable.DEFAULT = prettytable.PLAIN_COLUMNS


In [3]:
import pandas as pd
import sqlite3

# Створення бази в файлі
conn = sqlite3.connect("my_database.db")

# Завантаження з Google Sheets
url = "https://docs.google.com/spreadsheets/d/1B9Biy7gYjstobq2CVD5o_rCIObEFTLypff5JsSQB1Ng/export?format=csv&id=1B9Biy7gYjstobq2CVD5o_rCIObEFTLypff5JsSQB1Ng&gid=0"
df = pd.read_csv(url)

# Запис у базу
df.to_sql("Users_TC", conn, if_exists="replace", index=False)

49817

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49817 entries, 0 to 49816
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id_order      49817 non-null  object
 1   id_user       49817 non-null  int64 
 2   status        49817 non-null  object
 3   date_created  49817 non-null  object
 4   amount        49817 non-null  object
 5   id_region     49817 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 2.3+ MB


In [6]:
%sql SELECT * FROM Users_TC LIMIT 5;

 * sqlite:///my_database.db
Done.


id_order,id_user,status,date_created,amount,id_region
7391a43e-142d-49f0-91bd-e03c23e8de48,87630,success,2023-09-21 19:06:17 UTC,2,1
87be7485-0dbc-4fc3-8f3d-ba8f46f3b1c7,87630,success,2023-09-21 19:40:17 UTC,"4,5",1
64caed80-ddc7-4f77-b156-7b2fc772b6b8,82145,fail,2023-09-21 09:11:05 UTC,2,1
9dd77e41-7bd0-4ae3-8248-39b3cc789413,82145,fail,2023-09-21 15:21:15 UTC,2,1
c07c0bd1-eea4-457b-b89d-89b5945d879e,82145,fail,2023-09-21 16:31:17 UTC,2,1


In [7]:
df.head()

,id_order,id_user,status,date_created,amount,id_region
0,7391a43e-142d-49f0-91bd-e03c23e8de48,87630,success,2023-09-21 19:06:17 UTC,2,1
1,87be7485-0dbc-4fc3-8f3d-ba8f46f3b1c7,87630,success,2023-09-21 19:40:17 UTC,"4,5",1
2,64caed80-ddc7-4f77-b156-7b2fc772b6b8,82145,fail,2023-09-21 09:11:05 UTC,2,1
3,9dd77e41-7bd0-4ae3-8248-39b3cc789413,82145,fail,2023-09-21 15:21:15 UTC,2,1
4,c07c0bd1-eea4-457b-b89d-89b5945d879e,82145,fail,2023-09-21 16:31:17 UTC,2,1


ТС = total spend, рахується на кожного юзера окремо. Це сумарні покупки користувача за весь час користування продуктом.

Написати SQL скрипт, що витягує всіх юзерів, ТС яких вище ніж середній ТС всіх юзерів в розрізі регіонів


In [9]:
# %sql select id_user, id_region, SUM(amount) as TS from Users_TC where status = 'success' GROUP BY id_user, id_region;

In [10]:
%sql select id_region, AVG(TS) from (select id_user, id_region, SUM(amount) as TS from Users_TC where status = 'success' GROUP BY id_region, id_user) group by id_region;

 * sqlite:///my_database.db
Done.


id_region,AVG(TS)
1,89.34296577946768
2,73.22299779573842
3,113.63375380600262
5,96.30819981149858
6,58.267857142857146


In [11]:
%sql SELECT id_user, id_region, TS FROM Users_TC u JOIN (SELECT id_region, AVG(TS) AS avg_ts FROM Users_TC GROUP BY id_region) r ON u.id_region = r.id_region WHERE u.TS > r.avg_ts



 * sqlite:///my_database.db
(sqlite3.OperationalError) no such column: TS
[SQL: SELECT id_user, id_region, TS FROM Users_TC u JOIN (SELECT id_region, AVG(TS) AS avg_ts FROM Users_TC GROUP BY id_region) r ON u.id_region = r.id_region WHERE u.TS > r.avg_ts]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [12]:
df['amount'] = df['amount'].str.replace(',', '.')

In [13]:
df['amount'] = df['amount'].astype('float')

In [14]:
df1 = df.groupby(['id_user', 'id_region']).sum('amount')

In [15]:
df2 = df.groupby('id_user').sum('amount')

In [17]:
df.dtypes

,0
id_order,object
id_user,int64
status,object
date_created,object
amount,float64
id_region,int64


In [18]:
import pandas as pd

# Фільтруємо лише успішні покупки
df_success = df[df['status'] == 'success']

# Крок 1: Рахуємо TS (total spend) для кожного користувача
user_ts = df_success.groupby(['id_user', 'id_region'], as_index=False)['amount'].sum()
user_ts.rename(columns={'amount': 'TS'}, inplace=True)

# Крок 2: Рахуємо середній TS по кожному регіону
region_avg_ts = user_ts.groupby('id_region', as_index=False)['TS'].mean()
region_avg_ts.rename(columns={'TS': 'avg_TS'}, inplace=True)

# Крок 3: Об’єднуємо таблиці, щоб порівняти TS кожного користувача зі середнім по регіону
merged = pd.merge(user_ts, region_avg_ts, on='id_region')

# Крок 4: Вибираємо користувачів, у яких TS > середнього по регіону
result = merged[merged['TS'] > merged['avg_TS']].sort_values(by=['id_region', 'TS'], ascending=[True, False])

# Перегляд результату
print(result)

      id_user  id_region      TS     avg_TS
6142    85856          1  7582.5  90.590114
6322    86097          1  2831.5  90.590114
414     19264          1  2600.0  90.590114
5827    85329          1  2040.0  90.590114
4308    81097          1  1840.0  90.590114
...       ...        ...     ...        ...
8172    87481          6    92.5  58.991071
3413    76320          6    80.0  58.991071
3964    79491          6    80.0  58.991071
4370    81387          6    80.0  58.991071
7424    87123          6    59.0  58.991071

[2272 rows x 4 columns]


In [19]:
result.id_user.nunique()

2147

In [20]:
region_avg_ts

,id_region,avg_TS
0,1,90.590114
1,2,74.283615
2,3,115.278382
3,5,97.748822
4,6,58.991071


In [ ]:
# Кількість унікальних регіонів для кожного користувача
user_region_counts = df.groupby('id_user')['id_region'].nunique().reset_index()
user_region_counts.rename(columns={'id_region': 'num_regions'}, inplace=True)

# Фільтруємо користувачів, які були у більше ніж одному регіоні
users_multiple_regions = user_region_counts[user_region_counts['num_regions'] > 1]

# Результат
print(f"Кількість користувачів у кількох регіонах: {len(users_multiple_regions)}")
print(users_multiple_regions.head())

Кількість користувачів у кількох регіонах: 1417
     id_user  num_regions
209     9292            2
233    10201            2
467    20372            2
883    33631            2
932    35705            2


In [ ]:
df_success.groupby('id_user')['id_region'].nunique()

,id_region
id_user,
51,1
161,1
186,1
292,1
333,1
...,...
87631,4
87632,3
87633,3


Оскільки одні і ті ж самі юзери можуть робити замовлення в різних регіонах, то в першу чергу потрібно згрупувати по регіонах, а вже потім по id_user.

In [ ]:
df_success_TS = df_success.groupby(['id_region', 'id_user']).sum('amount')
df_success_TS.groupby('id_region')['amount'].mean()

,amount
id_region,
1,90.590114
2,74.283615
3,115.278382
5,97.748822
6,58.991071


In [21]:
df_success.amount.sum()

np.float64(812492.0)

In [23]:
df.amount.sum()

np.float64(1048044.5)

In [24]:
df.amount.unique()

array([  2. ,   4.5,  12.5,  80. ,  15. , 200. ,   6.5,  27. ,   0. ,
        50. ])

In [26]:
from google.colab import drive
drive.mount('/content/drive')


df.to_csv('/content/drive/My Drive/Task 11.csv', index=False)

Mounted at /content/drive
